In [1]:
"""
To get real time data from noaa api and put it into csvs NEED TO CHANGE TO 51101
"""

import wget
import pandas as pd
urlStandardData= 'https://www.ndbc.noaa.gov/data/realtime2/51101.txt'
urlWaveData = 'https://www.ndbc.noaa.gov/data/realtime2/51101.spec'

#For Standard wave Data
standardData = wget.download(urlStandardData)
col_names = ['YY', 'MM', 'DD', 'hh', 'mm', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD', 'APD', 'MWD', 'PRES', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'PTDY', 'TIDE']
meteo_data = pd.read_csv(standardData, delim_whitespace=True, comment='#', names=col_names)
meteo_data.to_csv('standard_meteorological_data51101.csv', index=False)



#For Spectral Wave Data
waveData = wget.download(urlWaveData)
col_names = ['YY', 'MM', 'DD', 'hh', 'mm', 'WVHT', 'SwH', 'SwP', 'WWH', 'WWP', 'SwD', 'WWD', 'Steepness', 'APD', 'MWD']
spec_wave_data = pd.read_csv(waveData, delim_whitespace=True, comment='#', names=col_names)
spec_wave_data.to_csv('spec_wave_data51101.csv', index=False)

/var/folders/bx/0ltqbd_9301dnzlwkd_qvdv80000gn/T/ipykernel_31456/2767960026.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  meteo_data = pd.read_csv(standardData, delim_whitespace=True, comment='#', names=col_names)
/var/folders/bx/0ltqbd_9301dnzlwkd_qvdv80000gn/T/ipykernel_31456/2767960026.py:21: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  spec_wave_data = pd.read_csv(waveData, delim_whitespace=True, comment='#', names=col_names)


In [2]:
"""
Data preprocessing and combining all data into one csv w matching wave information and meteorological data
"""

import pandas as pd
import numpy as np

# Read the data
sdata = pd.read_csv('spec_wave_data51101.csv')
mdata = pd.read_csv('standard_meteorological_data51101.csv')

# Convert the date-time columns into a single 'datetime' column
sdata['datetime'] = pd.to_datetime(sdata[['YY', 'MM', 'DD', 'hh', 'mm']].rename(columns={'YY': 'year', 'MM': 'month', 'DD': 'day', 'hh': 'hour', 'mm': 'minute'}))
mdata['datetime'] = pd.to_datetime(mdata[['YY', 'MM', 'DD', 'hh', 'mm']].rename(columns={'YY': 'year', 'MM': 'month', 'DD': 'day', 'hh': 'hour', 'mm': 'minute'}))

# Drop the individual date and time columns now that 'datetime' is created
# Drop the matching columns and unnecessary columns from the meteorlogical dataset
sdata.drop(['YY', 'MM', 'DD', 'hh', 'mm'], axis=1, inplace=True)
mdata.drop(['YY', 'MM', 'DD', 'hh', 'mm', 'WVHT','APD','MWD','TIDE','PTDY','VIS'], axis=1, inplace=True)

cols_to_convert = ['WDIR', 'WSPD', 'GST', 'DPD', 'PRES', 'ATMP', 'WTMP', 'DEWP']


mdata.replace('MM', np.nan, inplace=True)
sdata.replace('MM', np.nan, inplace=True)

# Convert columns to numeric where necessary in both datasets
sdata_numeric_cols = ['WVHT', 'SwH', 'SwP', 'WWH', 'WWP','APD', 'MWD']
#WVHT,SwH,SwP,WWH,WWP,SwD,WWD,Steepness,APD,MWD
mdata_numeric_cols = ['WDIR', 'WSPD', 'GST', 'DPD', 'PRES', 'ATMP', 'WTMP', 'DEWP']

# Apply numeric conversion to the necessary columns, coercing errors to NaN
sdata[sdata_numeric_cols] = sdata[sdata_numeric_cols].apply(pd.to_numeric, errors='coerce')
mdata[mdata_numeric_cols] = mdata[mdata_numeric_cols].apply(pd.to_numeric, errors='coerce')

# Interpolate missing values (linear interpolation) for Dominant Wave Period 
mdata['DPD'] = mdata['DPD'].interpolate(method='linear')
# Forward fill to handle missing values at the start
mdata['DPD'] = mdata['DPD'].fillna(method='bfill')

# Merge the two datasets based on the 'datetime' column, keeping all rows from spec_wave_data
combined_data = pd.merge(sdata, mdata, on='datetime', how='left')

#Saved Data
combined_data.to_csv('51101combined_data.csv', index=False)




/var/folders/bx/0ltqbd_9301dnzlwkd_qvdv80000gn/T/ipykernel_31456/109954113.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mdata.replace('MM', np.nan, inplace=True)
/var/folders/bx/0ltqbd_9301dnzlwkd_qvdv80000gn/T/ipykernel_31456/109954113.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  mdata['DPD'] = mdata['DPD'].fillna(method='bfill')
